<a href="https://colab.research.google.com/github/harveydogg/notebooks/blob/main/BinarySampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Walkthrough: Binary sampling from small populations
Version: Oct-25-2022<br>
Author: Justin Harvey

# Background

This analysis was motivated by a specific practical problem. We begin by explaining what this analysis is <i>for</i>, and why it was deemed <i>necessary</i>.

<u>What For</u>:<br>
Suppose we have a jar known to contain a thousand marbles, each red or blue, and would like to know their proportion. However, for practical reasons we are unable to examine all of them, so we need to select a random sample and use this to estimate. Moreover — again, for practical reasons — our goal is to examine as few marbles as possible, while still achieving an estimate we deem adequate. (Note: The original problem context was software evaluation, but marbles are easier to visualize.)

Naturally there will be a trade-off between "as few as possible" and "adequate estimate." And depending on the actual context, the way these goals are balanced may involve qualitative considerations best handled on a case-by-case basis. So our aim here is not final analysis; we merely seek to quantify that trade-off. 

<u>Why Necessary</u>:<br>
At first glance, our marble problem sounds like a standard Stats 101 example. But it has two features that, together, make it non-standard:

1. Sample selection is done without replacement.
2. The population $(N = 1,000)$ is small.

Mathematically our problem is most reminiscent of the binomial distribution, which characterizes repeat trials of an experiment also having exactly two possible outcomes. (For example, flipping a coin multiple times.) However, in binomial distributions the trials are <i>independent</i>, meaning the probabilities of the two outcomes remain constant from one trial to the next. Our problem does not have that feature. As we draw our sample, each individual is removed from the population, which means that each successive selection is drawn from a slightly altered population. (Think poker hands: the deck composition changes as each card is dealt.)

Of course, in practice this feature of the binomial distribution does not prevent its widespread applicablity to situations involving random sampling. (For example, opinion polls.) Provided the population is enormous relative to the sample, even the first and final selections can be treated as coming from the "same" population. (For reference: <a href="https://www.janda.org/c10/Lectures/topic05/GallupFAQ.htm#size" target="_blank">Gallup polls</a> typically sample 1,000 from a population of 187 million.) And given the mathematical cleanliness of the binomial distribution, this simplifying assumption is usually desirable.

But this is where (2) comes in. If the population is only, say, between a few hundred and a few thousand, then one might need to sample between 20% and 50% of the population to achieve a satisfactory estimate. In such cases, clearly we can't just assume the composition of the population stays constant throughout the process of selection. So we can't use the binomial distribution as an approximation. (Well, perhaps we can? But if so, providing a <i>proof</i> that we can would be more challenging than just doing the exact calculations. So for us it wouldn't be a <i>simplifying</i> assumption.)

# Scope and Strategy

Although we shall stick to our marbles, the following analysis and algorithms can be applied to a wide range of cases. However, there is one crucial restriction:

- <i>The variable must be binary.</i>

For example, our variable is color and it has two possible values: every marble in the population is either red or blue, but not both. Of course, our analysis could be extended to categorical variables having more than two values, provided the number of categories is finite. But we do not attempt that extension here; all formulas and algorithms apply exlusively to the case of binary variables.

We should also note that, although we motivated this analysis by citing small population size, this is not a requirement. In fact, no matter how large the population, our combinatorial approach produces the exact calculations for which the binomial distribution provides approximation. The downside of all that brute force counting is unscalability. The $\Theta$ function (our main result) computes in seconds on a 2020 MacBook for N = 1,000; in minutes for N = 2,500; in hours for N = 10,000.

We proceed in two steps. In Part 1, we assume we already know the composition of the jar (i.e. population) and use this to calculate the probability of collecting various random samples. This foundation will help us tackle our motivating problem in Part 2, where we estimate the (unknown) composition of the jar, based on a random sample we have already collected.

###### Note: If you are executing any cells in this notebook, execute all of them and do so in order.

In [ ]:
import math

'''
Prior to Python 3.8 the math libary did not include an "n-choose-k" function.
Since we shall rely heavily on this function, for backwards compatibility we here
define it manually rather than using "math.comb(n, k)" from Python 3.8.

Note: As of this writing, the Colab platform by Google is still using Python 3.7.
'''

def choose(n, k):
    '''
    The standard mathemetical function, nCk, standardly read as "n-choose-k"
    and mathematically defined as nCk = n! / (n-k)!k!.
    '''
    if n < k:
        return 0
    else:
        return math.factorial(n) // (math.factorial(n - k) * math.factorial(k)) 

In [ ]:
# Quick check to confirm 'choose(n,k)' returns same values as 'math.comb(n,k)' for all n,k on interval [0,99]

# Note: Running this cell will throw name error for Python versions < 3.8. This is fine.

passed = 0
failed = 0

for n in range(100):
    for k in range(100):
        if choose(n, k) != math.comb(n, k):
            failed += 1
        else:
            passed += 1

print("Passed =", passed)
print("Failed =", failed)

Passed = 10000
Failed = 0


# Part 1: Calculating probabilities of samples from known jars

To begin, let us stipulate two definitions that will simplify our discussion. We call a jar of given size and composition a <i>specific jar</i>, and a sample of given size and composition a <i>specific sample</i>. For example, a jar containing 100 marbles, of which 75 are blue and 25 are red, is a specific jar. Likewise, a sample containing 10 marbles, of which 6 are blue and 4 are red, is a specific sample.

Now suppose we wish determine the probability of getting a specific sample from a specific jar — assuming, as we shall always do, that the sample is randomly selected. To determine this, we need to count the number of ways of getting the specific sample from the specific jar, and divide by the total number of ways of getting all samples of that size from the specific jar. In other words, if we define:<br><br>

<div style="padding-left: 50px;">
    $ N := $ total marbles in jar <br>
    $ B := $ blues in jar <br>
    $ R := $ reds in jar <br>
    $ s := $ sample size <br>
    $ b := $ blues in sample <br>
    $ r := $ reds in sample <br>
</div>

then the probability we seek can be calculated by:

$$ \large \frac{ {B \choose b} {R \choose r} }{N \choose s} $$

where, recall:

$$ {n \choose k} = \frac{n!}{(n-k)! \cdot k!} $$

yields the number of possible ways to select $ k $ items from $ n $ items — standardly read, "n choose k." This formulation has the virtue of making the procedure transparent. However, notice that it contains redundancies, since clearly $ N = B + R $ and $ s = b + r $. Thus, we can substitute $ R = N - B $ and $ r = s - b $ to produce a more efficient formulation of the desired probability, which for now we represent as a function of $b$ (i.e., treating $N$, $B$, and $s$ as givens):

$$ f(b) = \large \frac{ {B \choose b} {N - B \choose s - b} }{N \choose s} $$

Interestingly, we can observe that an alternative (but mathematically equivalent) way of calculating $f$'s denominator would be:

$$ \sum_{b=0}^{s} \left[ {B \choose b} {N - B \choose s - b} \right] $$

Although this representation looks more cumbersome, notice the summand is just $f$'s numerator. In other words, abbreviating $f$'s numerator as $\phi(b)$, we also have:

$$ f(b) = \frac{\phi(b)}{\sum_{b=0}^{s}\phi(b)} $$

Here, a vernacular translation might help make the formulation intuitive. "Count the number of ways to get the specific sample, and then divide by the sum: [ways to get a sample with zero blues] + [ways to get a sample with 1 blue] + [ways to get a sample with 2 blues] + ... + [ways to get a sample with all blues]." <i>(Again, here we are treating $N$, $B$, and $s$ as fixed.)</i>


###### Code for these functions:

In [ ]:
def ways_to_get_specific_sample_from_specific_jar(jar_size, jar_blues, sample_size, sample_blues):
    '''
    This is our phi-function.
    In other words, the numerator of f.
    '''
    return choose(jar_blues, sample_blues) * choose(jar_size - jar_blues, sample_size - sample_blues)

In [ ]:
def ways_to_get_all_samples_of_given_size_from_specific_jar(jar_size, jar_blues, sample_size):
    '''
    This is the denominator of f.
    It sums our phi-function, over all possible values of b.
    
    Note: This is mathematically equivalent to choose(jar_size, sample_size).
    But for explanatory purposes that will become clear, we prefer this formulation.
    '''
    tally = 0
    for b in range(sample_size + 1):
        tally += ways_to_get_specific_sample_from_specific_jar(jar_size, jar_blues, sample_size, b)
    return tally    

In [ ]:
def probability_of_getting_specific_sample_from_specific_jar(jar_size, jar_blues, sample_size, sample_blues):
    '''
    This is our f.
    It just puts the numerator and denominator in their places.
    '''
    p = (
        ways_to_get_specific_sample_from_specific_jar(jar_size, jar_blues, sample_size, sample_blues)
        /
        ways_to_get_all_samples_of_given_size_from_specific_jar(jar_size, jar_blues, sample_size)
    )
    return p    

###### Example Calculation 1

In [ ]:
# Calculate probability of getting exactly 6 blues
# in random sample of 10 marbles
# from jar of 100 marbles with 75 blues

# Given knowns
N = 100
B = 75
s = 10

# Thing whose probability we're calculating
b = 6

solution = probability_of_getting_specific_sample_from_specific_jar(N, B, s, b)

print("Given a jar with", N, "marbles of which", B, "are blue and", N-B, "are red, "
      "if you randomly select", s, "marbles, the probability that exactly", b,
      "of them will be blue is", solution)

Given a jar with 100 marbles of which 75 are blue and 25 are red, if you randomly select 10 marbles, the probability that exactly 6 of them will be blue is 0.14714920688794267


# Part 2: Calculating probabilities of jars from known samples

Now we need to reverse our perspective. In the previous section, we considered the scenario in which we knew the composition of the source, from which we calculated the probability that a future random sample would have a particular composition. In this section, we consider the scenario in which we have already obtained a sample — so we know <i>its</i> composition — but we don't know the composition of the source from which it was taken. In short, we need to determine the probability <i>that a specific sample came from a specific jar</i>.

To fully appreciate the symmetry, note that in the previous section we envisioned:
- knowing $N$, $s$, $B$
- using them to determine the probability of <u>getting</u> a sample with $b$ blues

But now we are envisioning:
- knowing $N$, $s$, $b$
- using them to determine the probability that the sample <u>came from</u> a jar with $B$ blues

To calculate this, we need to count the number of ways of getting the specific sample from the specific jar, and then divide by the total number of ways of getting that specific sample from all possible jars (of that size). In other words, using placeholder $\psi(B)$ for the numerator, we need a function having the form:

$$ g(B) = \frac{\psi(B)}{\sum_{B=0}^{N}\psi(B)} $$

So then, what is $\psi(B)$? Well, the number of ways of getting a specific sample from a specific jar is:

$$ \large \frac{ {B \choose b} {N - B \choose s - b} }{N \choose s} $$

Look familiar? This formula is identical to our $\phi(b)$ from the previous section. There's no sleight of hand happening here — $\phi$ and $\psi$ are the "same" function; the only difference is which inputs we conceive as fixed and which as variable. (We use scarequotes out of an abundance of philosophical caution.) Symbolically, we might convey this formulaic coincidence as:

$$ \psi_{N,s,b}(B) = \large \frac{ {B \choose b} {N - B \choose s - b} }{N \choose s} \normalsize = \phi_{N,s,B}(b) $$

Metaphysical concerns aside, $\phi$ and $\psi$ are clearly the same function from a computational perspective. Thus we have already defined $g$'s numerator in previous code, so now we just need code for its denominator:


In [ ]:
def ways_to_get_specific_sample_from_all_jars_of_given_size(jar_size, sample_size, sample_blues):
    '''
    This is the denominator of g.
    It sums our phi-function, over all possible values of B.
    
    Note: Comparing this with f's denominator function above,
    the only difference is that now we are summing over all
    possible values of B=jar_blues instead of b=sample_blues.
    
    '''
    tally = 0
    for B in range(jar_size + 1):
        tally += ways_to_get_specific_sample_from_specific_jar(jar_size, B, sample_size, sample_blues)
    return tally


So our probability function $g(B)$ is:

In [ ]:
def probability_that_specific_sample_came_from_specific_jar(jar_size, jar_blues, sample_size, sample_blues):
    '''
    This is our g.
    It just puts the numerator and denominator in their places.
    '''
    p = (
        ways_to_get_specific_sample_from_specific_jar(jar_size, jar_blues, sample_size, sample_blues)
        /
        ways_to_get_specific_sample_from_all_jars_of_given_size(jar_size, sample_size, sample_blues)
    )
    return p

Here is an example calculation, using the same input values as Example Calculation 1:

###### Example Calculation 2.a

In [ ]:
# Given random sample of 10 from jar of 100 marbles,
# if the sample contains exactly 6 blues,
# calculate probability it came from jar with 75 blues.

# Given knowns
s = 10
N = 100
b = 6

# Thing whose probability we're calculating
B = 75

solution = probability_that_specific_sample_came_from_specific_jar(N, B, s, b)

print("Given a jar with", N, "marbles, if you randomly select", s,
     "of which", b, "are blue and", s-b,
      "are red, then the probability that the jar originally contained exactly", B,
     "blues is", solution)


Given a jar with 100 marbles, if you randomly select 10 of which 6 are blue and 4 are red, then the probability that the jar originally contained exactly 75 blues is 0.016026151245221477


Of course, in practice we will rarely be interested in the probability that the jar contained some <i>exact</i> number of blues. Instead, we will typically be interested in the probability that its number of blues was in some <i>interval</i> $[L, U]$, denoted by lower and upper bounds $L$ and $U$. Let us call this corresponding function $h$, which will be:

$$ h(B)_{[L,U]} = \frac{\sum_{B=L}^{U}\psi(B)}{\sum_{B=0}^{N}\psi(B)} $$

Since we have already defined $h$'s denominator in code above, we need only define its numerator:

In [ ]:
def ways_to_get_specific_sample_from_interval_of_jars(jar_size, lower, upper, sample_size, sample_blues):
    '''
    This is the numerator of h.
    It sums our phi-function over values of B on the interval [L, U].
    '''
    tally = 0
    for B in range(lower, upper + 1):
        tally += ways_to_get_specific_sample_from_specific_jar(jar_size, B, sample_size, sample_blues)
    return tally

So our probability function $h(B)_{[L,U]}$ is:

In [ ]:
def probability_that_specific_sample_came_from_interval_of_jars(jar_size, lower, upper, sample_size, sample_blues):
    '''
    insert docstring
    '''
    p = (
        ways_to_get_specific_sample_from_interval_of_jars(jar_size, lower, upper, sample_size, sample_blues)
        /
        ways_to_get_specific_sample_from_all_jars_of_given_size(jar_size, sample_size, sample_blues)
    )
    return p

Here is an example calculation using the same input values as Example 2.a, but replacing the specific value $B=75$ with the interval $B\in[73, 77]$:

###### Example Calculation 2.b

In [ ]:
# Given random sample of 10 from jar of 100,
# if the sample contains exactly 6 blues,
# calculate probability it came from jar with 73-77 blues.

# Given knowns
s = 10
N = 100
b = 6

# Thing whose probability we're calculating
L = 73
U = 77

solution = probability_that_specific_sample_came_from_interval_of_jars(N, L, U, s, b)

print("Given a jar with", N, "marbles, if you randomly select", s,
     "of which", b, "are blue and", s-b,
      "are red, then the probability that the jar originally contained between", L,
     "and", U, "blues is", solution)

Given a jar with 100 marbles, if you randomly select 10 of which 6 are blue and 4 are red, then the probability that the jar originally contained between 73 and 77 blues is 0.08007262572574683


Now up to this point, we have been setting our inputs $B,L,U$ arbitrarily. In practice, however, after we have drawn a random sample, the interval whose probability we'll be interested in will have the form $B^* \pm \Delta$, where the prudent choice for $B^*$ will be dictated rather trivially by the actual number of blues in the sample.

For example, supposing we got a random sample of 10 from a jar of 100, if our sample contained 6 blues and 4 reds, then the obvious "best guess" for $B$'s true value would be 60. In other words, the natural choice for $B^*$ will be determined by:

$$ B^* = b \cdot \left( \frac{N}{s} \right) $$

In the code, we use 'B_guess' or 'Bg' instead of $B^*$:

In [ ]:
def B_guess(jar_size, sample_size, sample_blues):
    '''
    The no-brainer guess for B, based on known value of b.
    We will use this 'Bg' as the center of our interval.
    '''
    return int(sample_blues * (jar_size / sample_size))

Finally, we just have one remaining issue: How shall we choose $\Delta$'s value? In practice, it depends. Sometimes we may be interested in some pre-specified interval. More often, however, we will be interested in finding the smallest $\Delta$ value that satisfies some pre-established probability threshold $C$, which will typically be 90%, 95%, or 99%. Computationally, we can achieve this by initializing $\Delta$ = 1 and incrementing until we reach a value of $\Delta$ where the probability satisfies our desired threshold.

Mathematically, we can clarify this by defining one last function $\Theta$:

$$ \Theta_{N,s}(b,C) = (B^*,\Delta^*) $$

where:

$$ B^* = b \cdot \left( \frac{N}{s} \right) $$

and:

$$ \Delta^* = \min \{ \Delta : h(B)_{[B^*-\Delta,B^*+\Delta]} \ge C \} $$

Here is one possible implementation of $\Theta$:

In [ ]:
def theta(jar_size, sample_size, sample_blues, confidence):
    '''
    insert docstring'''
    
    Bg = B_guess(jar_size, sample_size, sample_blues)
    
    delta = 1
    p = probability_that_specific_sample_came_from_interval_of_jars(jar_size, Bg - delta, Bg + delta, sample_size, sample_blues)
       
    while p < confidence:
        delta +=1
        p = probability_that_specific_sample_came_from_interval_of_jars(jar_size, Bg - delta, Bg + delta, sample_size, sample_blues)
    else:
        Dg = delta
        
    return (Bg, Dg)

###### Example Calculation 2.c

In [ ]:
# Given random sample of 10 from jar of 100,
# if the sample contains exactly 6 blues,
# calculate (Bg, Dg) such that:
# there is 95% probability the number of jar blues
# was on interval Bg +/- Dg.

# Given knowns
N = 100
s = 10
b = 6
C = 0.95

pair = theta(N, s, b, C)

minimum = pair[0] - pair[1]
maximum = pair[0] + pair[1]

print("Given a jar with", N, "marbles, if you randomly select", s,
     "of which", b, "are blue and", s-b,
      "are red, then there is a", C, "probability that the jar originally contained between", minimum,
     "and", maximum, "blues.")

Given a jar with 100 marbles, if you randomly select 10 of which 6 are blue and 4 are red, then there is a 0.95 probability that the jar originally contained between 35 and 85 blues.


###### Your sandbox for Theta calculations

In [ ]:
# Replace values with your desired inputs
N = 1000
s = 100
b = 60
C = 0.95

pair = theta(N, s, b, C)

minimum = pair[0] - pair[1]
maximum = pair[0] + pair[1]

print("Given a jar with", N, "marbles, if you randomly select", s,
     "of which", b, "are blue and", s-b,
      "are red, then there is a", C, "probability that the jar originally contained between", minimum,
     "and", maximum, "blues.")

Given a jar with 1000 marbles, if you randomly select 100 of which 60 are blue and 40 are red, then there is a 0.95 probability that the jar originally contained between 510 and 690 blues.
